## Production Planning Project: **Phase 3**


### Details

Full Name: **Hamed Araab**

Student Number: **9925003**


### Dependencies

First, we import the libraries that we are going to need later on.

- `pandas` and `numpy` for data manipulations.


In [129]:
import numpy as np
import pandas as pd

np.random.seed(69)

### Dataset

Here, we import the forecasting data from the previous phase.


In [130]:
phase_1_output = pd.read_excel(
    "../1. Demand Forecasting/output.xlsx",
    sheet_name="Data",
    index_col=0,
)

phase_1_output

,G1 Actual,G2 Actual,G3 Actual,G1 Forecast (SES),G2 Forecast (SES),G3 Forecast (SES),G1 Forecast (SMA),G2 Forecast (SMA),G3 Forecast (SMA),G1 Forecast (WMA),G2 Forecast (WMA),G3 Forecast (WMA),G1 Forecast (LR),G2 Forecast (LR),G3 Forecast (LR),G1 Forecast (ALR),G2 Forecast (ALR),G3 Forecast (ALR)
0,352.1,146.8,14.7,352.100000,146.800000,14.700000,352.100000,146.800000,14.700000,352.10000,146.800000,14.700000,395.830000,182.258571,17.584286,334.085280,135.515172,14.582346
1,469.3,171.7,16.2,352.100000,146.800000,14.700000,469.300000,171.700000,16.200000,469.30000,171.700000,16.200000,395.457368,182.059248,17.071203,426.822969,160.227161,17.641170
2,498.0,229.3,22.7,387.260000,154.270000,15.150000,498.000000,229.300000,22.700000,498.00000,229.300000,22.700000,395.084737,181.859925,16.558120,508.436307,230.524012,21.458910
3,476.5,220.7,19.2,420.482000,176.779000,17.415000,439.800000,182.600000,17.866667,460.21000,195.520000,19.150000,394.712105,181.660602,16.045038,487.321661,222.172337,18.686954
4,434.7,206.6,17.7,437.287400,189.955300,17.950500,481.266667,207.233333,19.366667,481.51000,213.480000,19.650000,394.339474,181.461278,15.531955,454.033856,207.979807,17.713604
5,405.4,185.6,14.7,436.511180,194.948710,17.875350,469.733333,218.866667,19.866667,459.90000,215.370000,19.150000,393.966842,181.261955,15.018872,413.492525,186.840937,15.739125
6,388.2,176.7,10.2,427.177826,192.144097,16.922745,438.866667,204.300000,17.200000,428.41000,198.920000,16.500000,393.594211,181.062632,14.505789,395.386796,177.730740,13.005491
7,311.8,176.4,14.7,415.484478,187.510868,14.905921,409.433333,189.633333,14.200000,402.66000,185.350000,13.050000,393.221579,180.863308,13.992707,349.805020,175.707265,13.510873
8,301.4,168.1,12.2,384.379135,184.177608,14.844145,368.466667,179.566667,13.200000,353.44000,178.330000,13.350000,392.848947,180.663985,13.479624,301.400000,168.100000,12.200000
9,291.4,161.1,10.5,359.485394,179.354325,14.050902,333.800000,173.733333,12.366667,321.88000,172.310000,12.550000,392.476316,180.464662,12.966541,291.400000,161.100000,10.500000


### Assumptions

Here are the assumptions that are made:


In [131]:
# the optimal forecasting methods according to the results of the first phase
selector = [
    "G1 Forecast (WMA)",
    "G2 Forecast (ALR)",
    "G3 Forecast (ALR)",
]

T = range(20, 26)  # periods
G = range(1, 4)  # product groups

# forecasts
f = (
    phase_1_output[selector]
    .rename(columns=dict(zip(selector, G)))
    .transpose()[T]
    .transpose()
    * 1000
).pipe(np.ceil)

f

,1,2,3
20,439880.0,165875.0,6628.0
21,434692.0,158965.0,5515.0
22,432838.0,153043.0,5006.0
23,434803.0,147120.0,4553.0
24,434192.0,131959.0,4371.0
25,434104.0,156018.0,4917.0


In [132]:
p = dict(zip(G, [2.4e4 * 25, 1.6e4 * 25, 1e4 * 25]))  # production (units)
st = dict(zip(G, [0.05 * 12, 0.03 * 12, 0.02 * 12]))  # startup time (months)
sc = dict(zip(G, [1e10, 8e9, 6e9]))  # startup cost (tomans)

# inventory surplus cost (tomans)
isc = dict(zip(G, [2.3e6 / 12, 3.1e6 / 12, 5.6e6 / 12]))

### Joint Production Feasibility

Here, we check if joint production is feasible under the aforementioned
assumptions. If the final value is less than 1, joint production will be
feasible.


In [133]:
# average demand over all product groups (units)
D_avg = f.stack().mean()

# demand (units)
D = dict(zip(G, [np.ceil(D_avg * p[g] / sum(p[g] for g in G)) for g in G]))

sum(D[g] / p[g] for g in G)

0.47393583333333333

### Joint Production Cycle Time

Now that we know that joint production is feasible, we calculate the joint
production cycle time.


In [134]:
# optimal cycle time (months)
CT_star = np.sqrt(
    2 * sum(sc[g] for g in G) / sum(isc[g] * D[g] * (1 - D[g] / p[g]) for g in G)
)

# minimum cycle time (months)
CT_min = sum(st[g] for g in G) / (1 - sum(D[g] / p[g] for g in G))

CT = max(CT_star, CT_min)  # cycle time (months)

CT_star, CT_min, CT

(1.03785685176601, 2.2810905513744366, 2.2810905513744366)

### Master Schedule


#### Parameters

We calculate the Master Schedule inputs. First, we use a weekly time frame in
the forecasts. Secondly, we calculate each order as a random number based on the
first forecast and the current week. Thirdly, we assume that safety stocks are
5% of the demands. Lastly, we calculate the lot sizes using the demands and
joint production cycle time.


In [135]:
W = range(1, len(T) * 4 + 1)

forecast = dict(
    zip(
        G,
        [dict(zip(W, [np.ceil(f[g][t] / 4) for t in T for _ in range(4)])) for g in G],
    )
)

order = dict(
    zip(
        G,
        [
            dict(
                zip(
                    W,
                    [
                        np.ceil(
                            forecast[g][1] * (np.random.random() * 0.4 + 0.8) / w**0.2
                        )
                        for w in W
                    ],
                )
            )
            for g in G
        ],
    )
)

safety_stock = dict(zip(G, [np.ceil(D[g] * 0.05) for g in G]))
lot_size = dict(zip(G, [np.ceil(D[g] * CT) for g in G]))

lot_size

{1: 216218.0, 2: 144145.0, 3: 90092.0}

#### Solver

Here, we define a function that solves a master schedule with the given
parameters.


In [136]:
def generate_master_schedule(
    forecast,
    order,
    on_hand,
    demand_time_fence,
    lot_size,
    safety_stock,
):
    forecast = {0: 0, **forecast}
    order = {0: 0, **order}

    demand = {
        0: 0,
        **{
            w: order[w] if w <= demand_time_fence else max(forecast[w], order[w])
            for w in W
        },
    }

    MPS = {0: 0, **{w: 0 for w in W}}
    ATP = {0: 0, **{w: 0 for w in W}}

    PoH = {0: on_hand}

    for w in W:
        while PoH[w - 1] - demand[w] + MPS[w] < safety_stock:
            MPS[w] += lot_size

        PoH[w] = PoH[w - 1] - demand[w] + MPS[w]

    for w in W:
        if MPS[w] == 0:
            continue

        ATP[w] = MPS[w]

        for v in range(w, W.stop):
            if v != w and MPS[v] != 0:
                break

            ATP[w] -= order[v]

    return pd.DataFrame(
        {
            "Forecast": forecast,
            "Order": order,
            "Demand": demand,
            "PoH": PoH,
            "ATP": ATP,
            "MPS": MPS,
        }
    ).transpose()

#### Results


In [137]:
ms_1 = generate_master_schedule(
    forecast=forecast[1],
    order=order[1],
    on_hand=safety_stock[1],
    demand_time_fence=6,
    lot_size=lot_size[1],
    safety_stock=safety_stock[1],
)

ms_1

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Forecast,0.0,109970.0,109970.0,109970.0,109970.0,108673.0,108673.0,108673.0,108673.0,108210.0,...,108701.0,108701.0,108548.0,108548.0,108548.0,108548.0,108526.0,108526.0,108526.0,108526.0
Order,0.0,101008.0,107570.0,82990.0,92990.0,81660.0,69276.0,62743.0,59740.0,75777.0,...,70151.0,58829.0,61655.0,53822.0,56719.0,59315.0,60527.0,54358.0,53500.0,62392.0
Demand,0.0,101008.0,107570.0,82990.0,92990.0,81660.0,69276.0,108673.0,108673.0,108210.0,...,108701.0,108701.0,108548.0,108548.0,108548.0,108548.0,108526.0,108526.0,108526.0,108526.0
PoH,4740.0,119950.0,12380.0,145608.0,52618.0,187176.0,117900.0,9227.0,116772.0,8562.0,...,6483.0,114000.0,5452.0,113122.0,220792.0,112244.0,219936.0,111410.0,219102.0,110576.0
ATP,0.0,7640.0,0.0,40238.0,0.0,2539.0,0.0,0.0,80701.0,0.0,...,0.0,95734.0,0.0,162396.0,100184.0,0.0,101333.0,0.0,100326.0,0.0
MPS,0.0,216218.0,0.0,216218.0,0.0,216218.0,0.0,0.0,216218.0,0.0,...,0.0,216218.0,0.0,216218.0,216218.0,0.0,216218.0,0.0,216218.0,0.0


In [138]:
ms_2 = generate_master_schedule(
    forecast=forecast[2],
    order=order[2],
    on_hand=safety_stock[2],
    demand_time_fence=6,
    lot_size=lot_size[2],
    safety_stock=safety_stock[2],
)

ms_2

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Forecast,0.0,41469.0,41469.0,41469.0,41469.0,39742.0,39742.0,39742.0,39742.0,38261.0,...,36780.0,36780.0,32990.0,32990.0,32990.0,32990.0,39005.0,39005.0,39005.0,39005.0
Order,0.0,37048.0,38661.0,32388.0,30016.0,28078.0,29133.0,33582.0,27698.0,28652.0,...,22506.0,19947.0,23133.0,26109.0,20192.0,22460.0,19769.0,18557.0,18004.0,18910.0
Demand,0.0,37048.0,38661.0,32388.0,30016.0,28078.0,29133.0,39742.0,39742.0,38261.0,...,36780.0,36780.0,32990.0,32990.0,32990.0,32990.0,39005.0,39005.0,39005.0,39005.0
PoH,3160.0,110257.0,71596.0,39208.0,9192.0,125259.0,96126.0,56384.0,16642.0,122526.0,...,41548.0,4768.0,115923.0,82933.0,49943.0,16953.0,122093.0,83088.0,44083.0,5078.0
ATP,0.0,6032.0,0.0,0.0,0.0,25654.0,0.0,0.0,0.0,41413.0,...,0.0,0.0,52251.0,0.0,0.0,0.0,68905.0,0.0,0.0,0.0
MPS,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,...,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0


In [139]:
ms_3 = generate_master_schedule(
    forecast=forecast[3],
    order=order[3],
    on_hand=safety_stock[3],
    demand_time_fence=6,
    lot_size=lot_size[3],
    safety_stock=safety_stock[3],
)

ms_3

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Forecast,0.0,1657.0,1657.0,1657.0,1657.0,1379.0,1379.0,1379.0,1379.0,1252.0,...,1139.0,1139.0,1093.0,1093.0,1093.0,1093.0,1230.0,1230.0,1230.0,1230.0
Order,0.0,1497.0,1195.0,1558.0,1280.0,1426.0,1342.0,1074.0,1041.0,1133.0,...,821.0,1122.0,1122.0,856.0,1019.0,842.0,1074.0,815.0,947.0,855.0
Demand,0.0,1497.0,1195.0,1558.0,1280.0,1426.0,1342.0,1379.0,1379.0,1252.0,...,1139.0,1139.0,1122.0,1093.0,1093.0,1093.0,1230.0,1230.0,1230.0,1230.0
PoH,1975.0,90570.0,89375.0,87817.0,86537.0,85111.0,83769.0,82390.0,81011.0,79759.0,...,72586.0,71447.0,70325.0,69232.0,68139.0,67046.0,65816.0,64586.0,63356.0,62126.0
ATP,0.0,63994.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MPS,0.0,90092.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Material Requirements Planning


#### Solver

In this section, we define a function that solves an MRP problem with the given
parameters.


In [140]:
def plan_material_requirements(
    gross_requirements,
    scheduled_receipts,
    on_hand,
    lot_size,
    lead_time,
    safety_stock,
):
    gross_requirements = {0: 0, **gross_requirements}
    scheduled_receipts = {0: 0, **scheduled_receipts}
    PoH = {0: on_hand}
    net_requirements = {0: 0, **{w: 0 for w in W}}
    planned_order_receipt = {0: 0, **{w: 0 for w in W}}
    planned_order_release = {0: 0, **{w: 0 for w in W}}

    for w in W:
        next_on_hand = PoH[w - 1] + scheduled_receipts[w] - gross_requirements[w]

        if next_on_hand < safety_stock:
            if w <= lead_time:
                raise Exception("Cannot release an order before the first period.")

            difference = safety_stock - next_on_hand

            if difference > safety_stock:
                net_requirements[w] = difference
            else:
                net_requirements[w] = 0

            while next_on_hand + planned_order_receipt[w] < safety_stock:
                planned_order_receipt[w] += lot_size

            PoH[w] = next_on_hand + planned_order_receipt[w]
            planned_order_release[w - lead_time] = planned_order_receipt[w]
        else:
            net_requirements[w] = 0
            PoH[w] = next_on_hand

    return pd.DataFrame(
        {
            "Gross Requirements": gross_requirements,
            "Scheduled Receipts": scheduled_receipts,
            "PoH": PoH,
            "Net Requirements": net_requirements,
            "Planned Order Receipt": planned_order_receipt,
            "Planned Order Release": planned_order_release,
        }
    ).transpose()

#### Results


In [141]:
mrp_1 = plan_material_requirements(
    gross_requirements=ms_1.transpose()["MPS"][1:],
    scheduled_receipts=dict(zip(W, [lot_size[1] if w == 1 else 0 for w in W])),
    on_hand=safety_stock[1],
    lot_size=lot_size[1],
    lead_time=1,
    safety_stock=safety_stock[1],
)

mrp_1

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Gross Requirements,0.0,216218.0,0.0,216218.0,0.0,216218.0,0.0,0.0,216218.0,0.0,...,0.0,216218.0,0.0,216218.0,216218.0,0.0,216218.0,0.0,216218.0,0.0
Scheduled Receipts,0.0,216218.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PoH,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,...,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0,4740.0
Net Requirements,0.0,0.0,0.0,216218.0,0.0,216218.0,0.0,0.0,216218.0,0.0,...,0.0,216218.0,0.0,216218.0,216218.0,0.0,216218.0,0.0,216218.0,0.0
Planned Order Receipt,0.0,0.0,0.0,216218.0,0.0,216218.0,0.0,0.0,216218.0,0.0,...,0.0,216218.0,0.0,216218.0,216218.0,0.0,216218.0,0.0,216218.0,0.0
Planned Order Release,0.0,0.0,216218.0,0.0,216218.0,0.0,0.0,216218.0,0.0,216218.0,...,216218.0,0.0,216218.0,216218.0,0.0,216218.0,0.0,216218.0,0.0,0.0


In [142]:
mrp_2 = plan_material_requirements(
    gross_requirements=ms_2.transpose()["MPS"][1:],
    scheduled_receipts=dict(zip(W, [lot_size[2] if w == 1 else 0 for w in W])),
    on_hand=safety_stock[2],
    lot_size=lot_size[2],
    lead_time=1,
    safety_stock=safety_stock[2],
)

mrp_2

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Gross Requirements,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,...,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0
Scheduled Receipts,0.0,144145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PoH,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,...,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0,3160.0
Net Requirements,0.0,0.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,...,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0
Planned Order Receipt,0.0,0.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,...,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0
Planned Order Release,0.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,...,0.0,144145.0,0.0,0.0,0.0,144145.0,0.0,0.0,0.0,0.0


In [143]:
mrp_3 = plan_material_requirements(
    gross_requirements=ms_3.transpose()["MPS"][1:],
    scheduled_receipts=dict(zip(W, [lot_size[3] if w == 1 else 0 for w in W])),
    on_hand=safety_stock[3],
    lot_size=lot_size[3],
    lead_time=1,
    safety_stock=safety_stock[3],
)

mrp_3

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Gross Requirements,0.0,90092.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scheduled Receipts,0.0,90092.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PoH,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,...,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0,1975.0
Net Requirements,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Planned Order Receipt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Planned Order Release,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Output

Finally, we save the MPS and MRP results in an excel file.


In [144]:
with pd.ExcelWriter("./output.xlsx") as writer:
    ms_1.to_excel(writer, sheet_name="MPS - G1")
    ms_2.to_excel(writer, sheet_name="MPS - G2")
    ms_3.to_excel(writer, sheet_name="MPS - G3")
    mrp_1.to_excel(writer, sheet_name="MRP - G1")
    mrp_2.to_excel(writer, sheet_name="MRP - G2")
    mrp_3.to_excel(writer, sheet_name="MRP - G3")